<a href="https://colab.research.google.com/github/JFSS20000/01MIAR04/blob/main/Model_ResUnet_JOSE_FERNANDO_SARMIENTO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Instala la librería
!pip install keras-unet-collection
!pip install --upgrade keras-unet-collection

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from keras_unet_collection import models

from tensorflow import keras
from keras.preprocessing.image import load_img
from keras import layers


In [4]:
def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model

In [5]:
# Llamar a la función get_model con una imagen de 160x160
img_size = (160, 160)
num_classes = 4
road_model = get_model(img_size, num_classes)
print(road_model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 160, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 80, 80,    │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 80, 80,    │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 80, 80,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 80, 80,    │          0 │ activation[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d    │ (None, 80, 80,    │      2,400 │ activation_1[0][… │
│ (SeparableConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 80, 80,    │        256 │ separable_conv2d… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 80, 80,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_1  │ (None, 80, 80,    │      4,736 │ activation_2[0][… │
│ (SeparableConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 80, 80,    │        256 │ separable_conv2d… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 40, 40,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 40, 40,    │      2,112 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 40, 40,    │          0 │ max_pooling2d[0]… │
│                     │ 64)               │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 40, 40,    │          0 │ add[0][0]         │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_2  │ (None, 40, 40,    │      8,896 │ activation_3[0][… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 40, 40,    │        512 │ separable_conv2d… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 40, 40,    │          0 │ batch_normalizat

 Total params: 2,059,268 (7.86 MB)

 Trainable params: 2,055,492 (7.84 MB)

 Non-trainable params: 3,776 (14.75 KB)

None


In [83]:
WEIGHTS_PATH_160 = 'road_segmentation_160_160.h5'
try:
    # Carga los pesos del archivo .h5
    road_model.load_weights(WEIGHTS_PATH_160)
    print("✅ Pesos 160x160 cargados exitosamente.")

except FileNotFoundError:
    print(f"❌ Archivo de pesos no encontrado en la ruta: {WEIGHTS_PATH_160}. Debes descargarlo primero.")
except ValueError:
    print("❌ Error: La arquitectura del modelo (160x160) no coincide con la de los pesos.")
except Exception as e:
    print(f"❌ Ocurrió un error inesperado al cargar los pesos 160x160: {e}")

❌ Error: La arquitectura del modelo (160x160) no coincide con la de los pesos.


In [81]:
# Definir arquitectura U-Net
#     320x320 píxeles
#     3 canales de entrada
#     1 canal de salida
# (ej. binario: carretera/no carretera)

IMG_SIZE = (320, 320)
NUM_CLASSES = 4

road_model = models.unet_2d(
    #input_size=(320, 320, 3),
    #filter_num=[32, 64, 128, 256, 512],   # Número de filtros por nivel
    #n_labels=1,                           # Una clase de salida (la carretera)
    # stack_num=2,                         # Doble convolución en cada bloque - Removed incorrect parameter
    #depth=5,                              # Use depth to specify the number of pooling/downsampling levels
    input_size=IMG_SIZE + (3,),      # (320, 320, 3)
    filter_num=[32, 64, 128, 256, 512], # Filtros inferidos
    n_labels=NUM_CLASSES,
    activation='ReLU',
    #output_activation='Sigmoid',          # Para segmentación binaria
    output_activation='Softmax',           # Para segmentación multiclase
    #output_padding='same',
    #use_bias=True,
    batch_norm=True,
    pool=True,
    unpool=True,
    name='unet_road_320'
)

# Compilación
road_model.compile(
    #optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    optimizer=tf.keras.optimizers.Adam(),
    #loss='binary_crossentropy',
    loss='categorical_crossentropy' if NUM_CLASSES > 1 else 'binary_crossentropy',
    metrics=['accuracy']
)

# Modelo definido, aún no tiene pesos entrenados.

In [82]:
WEIGHTS_PATH = 'road_segmentation_320_320.h5'
try:
    # Carga los pesos del archivo .h5
   # keras.models.load_model('res_unet_carreteras.h5')
    model1.load_weights(WEIGHTS_PATH)
    print("✅ Pesos cargados exitosamente.")

except FileNotFoundError:
    print("❌ Archivo de pesos no encontrado. Debes descargarlo primero.")
except ValueError:
    print("❌ Error: La arquitectura del modelo no coincide con la de los pesos.")

❌ Error: La arquitectura del modelo no coincide con la de los pesos.
